# Installing and Configuring a Slurm Master Instance

In [ ]:
hostname

**Change hostname** (if not already set)

In [ ]:
hostnamectl set-hostname slurmmaster

In [ ]:
systemctl restart network

In [ ]:
hostname

vim /etc/hosts

127.0.0.1 localhost

**Setup NTPD**

In [ ]:
yum install ntp

In [ ]:
systemctl enable ntpd.service

In [ ]:
systemctl start ntpd

**Setup Global Users**

Check if *slurm* or *munge* users already exist.  If they do, make sure their GID and UID are the same on all nodes.  If they don't, create them.

In [ ]:
cat /etc/passwd | grep slurm

In [ ]:
cat /etc/passwd | grep munge

Pick unused UIDs and GIDs.

In [ ]:
export MUNGEUID=991

In [ ]:
export SLURMUID=1001

In [ ]:
export MUNGEGID=985

In [ ]:
export SLURMGID=1001

In [ ]:
groupadd -g $MUNGEGID munge

In [ ]:
groupadd -g $SLURMGID slurm

In [ ]:
useradd  -m -c "Munge" -d /var/lib/munge -u $MUNGEUID -g munge -s /sbin/nologin munge

In [ ]:
useradd  -m -c "SLURM workload manager" -d /var/run/slurm -u $SLURMUID -g slurm -s /bin/bash slurm

In */etc/passwd* should see the following two entries:
```
munge:x:991:985:Munge:/var/lib/munge:/sbin/nologin
slurm:x:1001:1001:SLURM workload manager:/var/lib/slurm:/bin/bash
...
```

** Install and configure MariaDB **

In [ ]:
yum install mariadb-server mariadb-devel -y

In [ ]:
systemctl enable mariadb

In [ ]:
systemctl start mariadb

In [ ]:
mysql_secure_installation

Start a mysql shell:

In [ ]:
mysql -p

In [ ]:
MariaDB [(none)]> create database slurm_acct_db;

In [ ]:
MariaDB [(none)]> grant all on slurm_acct_db.* TO 'slurm'@'localhost';

**Configure and Enable SLURMDBD**

Create the following *slurmdbd.conf* file and place it in */etc/slurm/*
```
AuthType=auth/munge
DbdAddr=localhost
DbdHost=localhost
SlurmUser=slurm
DebugLevel=4
LogFile=/var/log/slurm/slurmdbd.log
PidFile=/var/run/slurmdbd.pid
StorageType=accounting_storage/mysql
StorageHost=localhost
StoragePass=somepasswd
StorageUser=slurm
StorageLoc=slurm_acct_db
```

In [ ]:
systemctl enable slurmdbd 

In [ ]:
systemctl start slurmdbd

** Install and Configure Munge **

In [ ]:
yum install epel-release

In [ ]:
yum install munge munge-libs munge-devel -y

After installing Munge, create a secret ket with it only on the Slurm server

In [ ]:
yum install rng-tools -y

In [ ]:
rngd -r /dev/urandom

In [ ]:
/usr/sbin/create-munge-key -r

In [ ]:
dd if=/dev/urandom bs=1 count=1024 > /etc/munge/munge.key

In [ ]:
chown munge: /etc/munge/munge.key

In [ ]:
chmod 400 /etc/munge/munge.key

Create an SSH pub key and add it to all nodes.

In [ ]:
ssh-keygen -t rsa

In [ ]:
cat ~/.ssh/id_rsa.pub | ssh root@nfs-1 'cat >> ~/.ssh/authorized_keys'

Copy the Munge key to all nodes you want to partake in the cluster

In [ ]:
scp /etc/munge/munge.key root@nfs-1:/etc/munge

Login on every node on the cluster and change the Munge key permissions and then start Munge

In [ ]:
chown -R munge: /etc/munge/ /var/log/munge/

In [ ]:
chmod 0700 /etc/munge/ /var/log/munge/

In [ ]:
systemctl enable munge

In [ ]:
systemctl start munge

Test Communication Between Munge Clients

In [ ]:
munge -n

In [ ]:
munge -n | unmunge

In [ ]:
munge -n | ssh nfs-1 unmunge

In [ ]:
remunge

** Install and Configure Slurm **

Install Slurm Dependencies:

In [ ]:
yum install -y rrdtool hwloc munge-devel munge-libs readline-devel perl-ExtUtils-MakeMaker openssl-devel pam-devel rpm-build perl-DBI perl-Switch munge mariadb-devel

Copy Slurm RPMs to all cluster nodes:

In [ ]:
scp -r /root/slurm_rpms nfs-1:/root/slurm_rpms

In [ ]:
rpm -Uvh /root/slurm_rpms/*.rpm

Create Slurm Home Dir (if needed)

In [ ]:
usermod -d /home/slurm slurm

In [ ]:
mkdir /home/slurm

In [ ]:
chmod -R slurm: /home/slurm

Open Ports

In [ ]:
firewall-cmd --permanent --zone=public --add-port=6817/udp

In [ ]:
firewall-cmd --permanent --zone=public --add-port=6817/tcp

In [ ]:
firewall-cmd --permanent --zone=public --add-port=6818/udp

In [ ]:
firewall-cmd --permanent --zone=public --add-port=6818/tcp

In [ ]:
firewall-cmd --permanent --zone=public --add-port=7321/udp

In [ ]:
firewall-cmd --permanent --zone=public --add-port=7321/tcp

In [ ]:
firewall-cmd --reload

Start Slurm on All Compute Nodes First

In [ ]:
systemctl start slurmd

Start the *slurmctld* on the Slurm Master Node

In [ ]:
systemctl enable slurmctld

In [ ]:
systemctl start slurmctld